# **Korean Summarizer Using Multiple Discriminators**

참조 : https://www.kaggle.com/shivamb/beginners-guide-to-text-generation-using-lstms

참조 : https://github.com/williamSYSU/TextGAN-PyTorch

* 2020년12월27일 v1.0 완전히 실패...
* Generator 새로 제작!

In [1]:
!pip install sentence-transformers==0.3.0
!pip install transformers==3.0.2
!pip install wikipedia
!pip install konlpy

     |████████████████████████████████| 61 kB 286 kB/s eta 0:00:01
  Created wheel for sentence-transformers: filename=sentence_transformers-0.3.0-py3-none-any.whl size=86752 sha256=723b38dcb4afe550bf3fb7b39d6e0e6e4c5633db23e40bdf542aca842c973177
  Stored in directory: /root/.cache/pip/wheels/3e/15/94/49bc84289d2c77b5059bca513f840c6006d4e2cc7f10275d49
Successfully built sentence-transformers
You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.
     |████████████████████████████████| 769 kB 1.3 MB/s eta 0:00:01


     |████████████████████████████████| 3.0 MB 5.8 MB/s eta 0:00:01     |███████████████████████         | 2.2 MB 5.8 MB/s eta 0:00:01
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.9.3
    Uninstalling tokenizers-0.9.3:
      Successfully uninstalled tokenizers-0.9.3
  Attempting uninstall: transformers
    Found existing installation: transformers 3.5.1
    Uninstalling transformers-3.5.1:
      Successfully uninstalled transformers-3.5.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
allennlp 1.2.2 requires transformers<3.6,>=3.4, but you have transformers 3.0.2 which is incompatible.
You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11685 sha256=7873764bdc32d816a0f153d958039d22f1f5bd13217de5988

In [2]:
# keras module for building LSTM 
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding, LSTM, Dense, Dropout
from keras.preprocessing.text import Tokenizer
from keras.callbacks import EarlyStopping
from keras.models import Sequential
from tensorflow.python.framework import tensor_shape
import keras.utils as ku 

# set seeds for reproducability
from tensorflow.random import set_seed
from numpy.random import seed
set_seed(2)
seed(1)

import pandas as pd
import numpy as np
import string, os 

import warnings
warnings.filterwarnings("ignore")
warnings.simplefilter(action='ignore', category=FutureWarning)

# 학습을 위한 데이터 준비

네이버 뉴스에서 아무거나 하나 Text를 얻어옴

이것을 '요약' 목표

In [4]:
org_text = """주호영 국민의힘 원내대표는 22일 고위공직자범죄수사처(공수처)법 개정과 가덕도 신공항 건설 등을 밀어붙이고 있는 문재인 정권과 더불어민주당을 향해 "이제 끝이 보인다"며 "짓밟힌 풀들이 아우성 치는 국민적 저항에 직면할 것"이라고 경고했다.
주 원내대표는 이날 자신의 페이스북에 "문재인 정권이 공수처법 개정을 위한 '군사작전'에 돌입하겠다고 엄포를 놓고 있다"며 "정의당을 끌어들이기 위해 꼼수 선거법에 묶어 '패스트트랙'이라는 불법·탈법으로 만들어낸 공수처법을 시행도 해보지 않고 고치려 하는 것"이라고 지적했다.
이어 주 원내대표는 "야당 원내대표인 제게 문재인 대통령은 사람 좋아보이는 표정으로 '공수처는 야당의 동의 없이는 절대 출범할 수 없는 것'이라고 얘기했고, 야당이 유엔 안보리 상임이사국처럼 공수처장 임명에 '비토권'을 행사할 수 있는데 무얼 걱정하느냐고, 여당 사람들이 우리를 속였다"며 "거짓말이라는 비난을 개의치 않는 사람들"이라고 꼬집었다.
주 원내대표는 "이해찬 전 민주당 대표가 얘기한 '민주당 20년 집권'의 토대가 올해 안에 완성된다"며 "탈원전과 동남권 신공항은 문 대통령이 대선 공약으로 내건 사업이니 여기에 불법이 있었다고 시비를 거는 것은 민주주의를 부정하는 것이라고 청와대 출신 윤건영 민주당 의원이 윽박지른다. 이제 '민주주의 없는 민주당'이 법위에 군림하는 '반민주'를 거리낌없이 획책하는 것"이라고 언급했다.
그러면서 주 원내대표는 "표를 얻기 위해 나라 곳간을 다 허물어뜨렸고, 재정 운용에서 신중함은 사라졌다"며 "괴물 공수처가 출범하면 공무원 누구나 대통령과 권력이 지시하는 범죄행위에 거리낌 없이 가담할 것이다. 청와대와 권부 요직에 앉아 불법으로 각종 이권을 챙기는 권력자들에 대한 사건이 불거져도 공수처가 사건을 가져가 버리면 그만"이라고 우려했다.
주 원내대표는 "문 대통령은 제게 '공수처는 고위 공직자들을 처벌하는 것인데 왜 야당이 반대하는지 이해할 수 없다'고 했는데, 그런 분이 청와대와 대통령 주변을 감시하는 특별감찰관은 취임 이후 지금까지 왜 임명하지 않았는가"라며 "공수처는 권력형 비리의 쓰레기 하치장, 종말 처리장이 될 것"이라고 비판했다.
문재인 정부를 향해 주 원내대표는 "문 대통령과 그 사도들은 법치가 미치지 않는 무오류의 화신이 될 것"이라며 "오류를 인정하지 않는 존재가 바로 신이며 그 아래에는 자신들의 지도자를 목숨바쳐 지킴으로서 정의를 실현하겠다는 추종자들로 넘쳐 난다. 공수처는 지도자의 신성을 인정하지 않는 세력을 정죄하는 수단으로 전락할 것"이라고 질타했다.
주 원내대표는 "저도 법조인이지만 대통령과 공수처장이 마음대로 검사들과 수사관들을 임명하는 이 끔찍한 사법기구가 어떤 일을 할지 두렵기만 하다"며 "공수처는 검찰과 경찰 위에 있는 사법기구로, 헌법과 법으로 독립성을 보장하는 검찰총장을 이렇게 핍박하는 정권이 공수처를 어떻게 운영할지 불을 보듯 뻔한 일"이라고 예측했다.
그러면서 주 원내대표는 "추미애 법무장관을 앞장 세워 윤석열 검찰의 권력 비리 수사를 저지하려다가 난관에 봉착하자 무슨 수를 써서라도 공수처를 출범시키려 한다. 공수처장 자리에는 추미애보다 더 한 막무가내 내 편을 앉힐 게 분명한 것"이라며 "문 정권의 파렴치와 오만함을 최전선에서 온 몸으로 겪어온 저로서는 민주당이 내일부터 국회에서 보일 행태가 환히 보인다. 180석의 민주당이 또 군사작전을 개시하면 그걸 누가 막겠는가"라고 성토했다.
주 원내대표는 "공수처법을 막을 힘이 우리 야당에게는 없다. 삭발하고 장외투쟁해 봐야 눈 하나 깜짝할 사람들이 아닌 것"이라며 "대란대치(大亂大治), 세상을 온통 혼돈 속으로 밀어넣고 그걸 권력 유지에 이용한다는 게 이 정권의 통치기술"이라고 규탄했다.
아울러 주 원내대표는 "권력은 바람, 국민은 풀이다. 바람이 불면 청보리 밭의 보리가 눕는다"며 "권력은 풀들이 다시는 일어서지 못하도록 풀을 짓밟지만 풀들은 다시 일어난다. 시인 김수영은 '바람보다 먼저 눕지만, 바람보다 먼저 일어나는' 민초의 힘을 노래했다"고 말했다.
마지막으로 주 원내대표는 "문재인 정권은 이제 곧 국회에서 광장에서 짓밟힌 풀들이 일어서서 아우성치는 모습을 지켜보게 될 것"이라며 "대란대치를 끝장내려는 국민적 저항에 직면할 것"이라고 거듭 강조했다.
"""

In [6]:
org_text

'\n주호영 국민의힘 원내대표는 22일 고위공직자범죄수사처(공수처)법 개정과 가덕도 신공항 건설 등을 밀어붙이고 있는 문재인 정권과 더불어민주당을 향해 "이제 끝이 보인다"며 "짓밟힌 풀들이 아우성 치는 국민적 저항에 직면할 것"이라고 경고했다.\n주 원내대표는 이날 자신의 페이스북에 "문재인 정권이 공수처법 개정을 위한 \'군사작전\'에 돌입하겠다고 엄포를 놓고 있다"며 "정의당을 끌어들이기 위해 꼼수 선거법에 묶어 \'패스트트랙\'이라는 불법·탈법으로 만들어낸 공수처법을 시행도 해보지 않고 고치려 하는 것"이라고 지적했다.\n이어 주 원내대표는 "야당 원내대표인 제게 문재인 대통령은 사람 좋아보이는 표정으로 \'공수처는 야당의 동의 없이는 절대 출범할 수 없는 것\'이라고 얘기했고, 야당이 유엔 안보리 상임이사국처럼 공수처장 임명에 \'비토권\'을 행사할 수 있는데 무얼 걱정하느냐고, 여당 사람들이 우리를 속였다"며 "거짓말이라는 비난을 개의치 않는 사람들"이라고 꼬집었다.\n주 원내대표는 "이해찬 전 민주당 대표가 얘기한 \'민주당 20년 집권\'의 토대가 올해 안에 완성된다"며 "탈원전과 동남권 신공항은 문 대통령이 대선 공약으로 내건 사업이니 여기에 불법이 있었다고 시비를 거는 것은 민주주의를 부정하는 것이라고 청와대 출신 윤건영 민주당 의원이 윽박지른다. 이제 \'민주주의 없는 민주당\'이 법위에 군림하는 \'반민주\'를 거리낌없이 획책하는 것"이라고 언급했다.\n그러면서 주 원내대표는 "표를 얻기 위해 나라 곳간을 다 허물어뜨렸고, 재정 운용에서 신중함은 사라졌다"며 "괴물 공수처가 출범하면 공무원 누구나 대통령과 권력이 지시하는 범죄행위에 거리낌 없이 가담할 것이다. 청와대와 권부 요직에 앉아 불법으로 각종 이권을 챙기는 권력자들에 대한 사건이 불거져도 공수처가 사건을 가져가 버리면 그만"이라고 우려했다.\n주 원내대표는 "문 대통령은 제게 \'공수처는 고위 공직자들을 처벌하는 것인데 왜 야당이 반대하는지 이해할 수 없다\'고 했는데, 그런 분이

한국어 문법체계에 따라 요약문을 생성하기 위해 한국어 문장 샘플을 준비

'한글 위키백과'에서 임의의 문장을 수집 함

In [7]:
#한국어 위키백과에서 스크랩핑

import wikipedia as wiki
wiki.set_lang('ko')

In [8]:
# '전래동화' 라는 keyword로 100개 page의 Text를 취득

def __search_from_wiki(question,max_rank):
    results = wiki.search(question,results=max_rank)
    print(results)
    contents = []
    for result in results:
        try:
            page = wiki.page(result)
            #print(f"Top wiki result: {page}")
            text = page.content
            ln = len(text)
            print(f'Collecting page : {page} , text length {str(ln)}')
            #if ln < 4000:
            #  contents.append(text)
            #else:
            #  contents.append(text[0:4000])
            contents.append(text)
        except Exception as ex:
          print(ex)
    return contents


ko_grammar_set_raw = __search_from_wiki("전래동화", 100)

print(f'전체 수집한 Page Count : {len(ko_grammar_set_raw)}')

['동화', '신 전래동화', '꾸러기 수비대', '아동 문학', '호시조라 미유키', '한국의 사찰', '해와 달이 된 오누이', '잠자는 숲속의 미녀', '거북', '이한갈', '정은찬', '김기두 (배우)', '옛날 옛적에 (애니메이션)', '계룡선녀전 (드라마)', '동요', '이상훈 (1976년)', '최홍일', '육진수 (격투기 선수)', '제비', '장석현 (연예인)', '유다미', '최지웅', '밀교 (불교)', '한다은', '박재훈 (배우)', '재희', '정미남', '안젤리나 다닐로바', '도깨비', '기탄교육', '국지용', '의왕백운호수축제', '박민경', '정정아', '콩딱쿵! 이야기 주머니', '윤기원 (배우)', '선녀와 나무꾼', '남생이', '콩쥐팥쥐 (동음이의)', '대장화홍련전', '토끼전', '미녀와 야수', '지대한', '이서휴게소', '은비까비의 옛날옛적에', '빨간 자전거', '콩쥐팥쥐전', '이설구', '아시리아인', '홍석연', '백조의 호수', '김덕현 (배우)', '티베트', '서예', '도교', '켈트 다신교', '금도끼 은도끼', '장화, 홍련 (동음이의)', '허구 국가', '노상현', '제네시스 (밴드)', '타이완의 문화', '도깨비 (동음이의)', '골디락스', '선녀강림', '손춘익', '자와어', '안동국제탈춤페스티벌', '윌리엄 버틀러 예이츠', '성덕대왕신종', '모모타로', '고려', '한국 문학', '라푼젤 (영화)', '토비트', '조선 후기의 문학', '송월동 동화마을', '홍석천', '계룡선녀전', '이집트', '응우옌 왕조', '아시아의 역사', '안지환', '외래어', '메이플 월드', '프랑스인', '스키타이족', '인도네시아', '이탈리아', 'MBC 창작동요제', '네버랜드', '일본 제국', '프랑크인', '김환영 (작가)', '바이킹', '드랑 나흐 오스텐', '조선', '오윤 (화가)', '원나라', '앱북']
"콩쥐팥쥐 (동음이의)" may refer to: 
콩쥐팥쥐


In [ ]:
ko_grammar_set_raw += __search_from_wiki("역사", 100)

print(f'전체 수집한 Page Count : {len(ko_grammar_set_raw)}')

In [9]:
# 간단한 전처리
def clean_text(txt):
    txt = txt.replace('\n','')
    txt = txt.replace('=','')    
    return txt 

ko_grammar_set_raw = [clean_text(x) for x in ko_grammar_set_raw]
print('Sample text : ')
print('--------------------------------------------------------------------------------------------')
print(ko_grammar_set_raw[50])
print('--------------------------------------------------------------------------------------------')

Sample text : 
--------------------------------------------------------------------------------------------
김덕현(본명은 김덕구, 1967년 4월 19일 ~ )은 대한민국의 배우이다. 충청남도 보령에서 태어나 청소년기를 보냈다. 서울예술전문대학 방송연예학과를 졸업했다. 1991년 KBS 14기 공채 탤런트로 데뷔했다. 학력 서울예술전문대학 방송연예학과 졸업 출연작  드라마 KBS2 단막드라마 《TV 손자병법》(1992년)KBS2 단막드라마 《신 손자병법》(1993년)KBS2 수목드라마 《전설의 고향》(1996년 6월 26일 ~ 1996년 9월 12일) ... 구미호/망자의 소원 역KBS2 월화 미니시리즈 《거짓말》(1998년 3월 20일 ~ 1998년 6월 2일)KBS2 드라마 《부부클리닉 사랑과 전쟁》(1999년 10월 22일 ~ 2009년 4월 17일)KBS2 월화 미니시리즈 《학교》(1999년 2월 22일 ~ 1999년 4월 13일) ... 선생님 역MBC 단막극 《MBC 베스트극장》(1999년)iTV 일일드라마 《해바라기 가족》(2001년)MBC 법정드라마 《실화극장 죄와 벌》(2003년)KBS2 주말연속극 《애정의 조건》 (2004년 3월 20일 ~ 2004년 10월 10일)SBS 일일드라마 《소풍가는 여자》 (2004년 5월 3일 ~ 2004년 10월 8일)KBS1 일일연속극 《금쪽같은 내 새끼》(2004년 6월 7일 ~ 2005년 2월 11일)KBS1 대하드라마 《불멸의 이순신》(2004년 9월 4일 ~ 2005년 8월 28일) ... 언복 역KBS1 TV소설 《고향역》(2005년 8월 29일 ~ 2006년 3월 18일) ... 최 계장 역KBS1 대하드라마 《서울 1945》(2006년 1월 7일 ~ 2006년 9월 10일) - 박헌영 비서 역KBS2 아침드라마 《아줌마가 간다》(2006년 11월 13일 ~ 2007년 5월 19일) - 김성태 역KBS2 아침드라마

문장으로 잘라 낸다

In [10]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [11]:
#Split the document into sentences
ko_grammar_sentences = []
for document in ko_grammar_set_raw:
    ko_grammar_sentences += nltk.sent_tokenize(document)

print("Num sentences:", len(ko_grammar_sentences))

Num sentences: 5729


In [12]:
ko_grammar_sentences[300]

"현재 기탄교육은 학습지로는 '기탄국어, 한글떼기, 기탄한글, 기탄수학, 기탄사고력수학, 영단어 암기 끝!, 기탄 한자 빨리따기, 기탄중국어' 등이 있다."

In [14]:
f = open("ko_sentence_set.txt", "a")
for t in ko_grammar_sentences:
    f.write(t+'\n')
f.close()

형태소 분리하여 모든 문장을 형태소 Code로 변환 한다.

In [13]:
from konlpy.tag import Twitter
twitter = Twitter()

print(twitter.pos(ko_grammar_sentences[305]))

[('백운호수', 'Noun'), ('에서', 'Josa'), ('의왕시', 'Noun'), ('와', 'Josa'), ('의왕시', 'Noun'), ('축제', 'Noun'), ('추진', 'Noun'), ('위원회', 'Noun'), ('가', 'Josa'), ('공동', 'Noun'), ('으로', 'Josa'), ('주최', 'Noun'), ('하여', 'Verb'), ('매년', 'Noun'), ('9월', 'Number'), ('에', 'Foreign'), ('이틀', 'Noun'), ('동안', 'Noun'), ('열리는', 'Verb'), ('축제', 'Noun'), ('이다', 'Josa'), ('.', 'Punctuation')]


In [15]:
# 형태소 Code table의 구성

_MAX_MORP_LENGTH = 128
_PADDING_CODE = 0  # padding code
_MISMATCH_CODE = 1 # mismatch word code               ex) @@@
_MISMATCH_WORD = '@@@' # 이거 아래에서 쓴다.

morpheme_table = {}
morp_code = _MISMATCH_CODE+1
morpheme_table['Pad'] = _PADDING_CODE 
morpheme_table['Mst'] = _MISMATCH_CODE 
for sentence in ko_grammar_sentences[:1000]:
    morphemes = twitter.pos(sentence)
    for (word,morp) in morphemes:
        if morp in morpheme_table:
            pass
        else:
            morpheme_table[morp] = morp_code
            morp_code += 1

In [16]:
print('Korean morpheme code table')
print('----------------------------------------------------------')
print('  Morpheme        Code')
print('')
for morp in morpheme_table.keys():
    print(f' {morp.ljust(15)}   {morpheme_table[morp]}')
print('----------------------------------------------------------')


Korean morpheme code table
----------------------------------------------------------
  Morpheme        Code

 Pad               0
 Mst               1
 Noun              2
 Punctuation       3
 Foreign           4
 Josa              5
 Verb              6
 Modifier          7
 Adjective         8
 Suffix            9
 Adverb            10
 Number            11
 Alpha             12
 Conjunction       13
 Determiner        14
 VerbPrefix        15
 Exclamation       16
 KoreanParticle    17
 Eomi              18
 ScreenName        19
 URL               20
----------------------------------------------------------


In [17]:
# morpheme 코드 변환기
def morpheme_encode(sentence):
    encode=[]
    morphemes = twitter.pos(sentence)
    for (word,morp) in morphemes:
        encode.append(_MISMATCH_CODE if word==_MISMATCH_WORD else morpheme_table[morp])
    return encode

code = morpheme_encode('야당이 수단으로 야당에게는 이라며 윤석열 허물어뜨렸고 수사를 군사작전을 하는 시비를 민주당 의원이 국민적 세워 화신이 나라 법위에 우려했다 될 이라며 운영할지 바람 짓밟지만 양자역학 양자역학 걱정하느냐고 누가 청와대 한다 엄포를 양자역학 양자역학 양자역학 편을 파렴치와 난관에 풀이다 대통령이 양자역학 저지하려다가')
print(f'Code length : {len(code)}')

Code length : 67


In [18]:
# 전체 형태소 코드로 변환
ko_grammar_set = []
for sentence in ko_grammar_sentences:
    code = morpheme_encode(sentence)
    if len(code) <= _MAX_MORP_LENGTH:
        ko_grammar_set.append(code + [_PADDING_CODE for i in range(_MAX_MORP_LENGTH-len(code))])

ko_grammar_set = np.asarray(ko_grammar_set)
ko_grammar_set.shape

(5637, 128)

In [19]:
from numpy import savetxt
# save to csv file
savetxt('ko_morpheme_set.csv', ko_grammar_set, delimiter=',')


# Dataset 전체 구성

In [20]:
from sentence_transformers import SentenceTransformer
from transformers import BertTokenizer
# embedder download...
embedder = SentenceTransformer('xlm-r-large-en-ko-nli-ststb')

wandb: WARNING W&B installed but not logged in.  Run `wandb login` or set the WANDB_API_KEY env variable.
100%|██████████| 1.80G/1.80G [08:32<00:00, 3.51MB/s] 


In [21]:
BATCH_SIZE = 64
BATCH_COUNT = 50

In [22]:
#dataset 다시 만듦

org_text_emb = embedder.encode([org_text])[0]
print(f'Text embedding shape : {org_text_emb.shape}')
dataset = []
for i in range(BATCH_COUNT):
    emb_batch_set = []
    cod_batch_set = []
    for j in range(BATCH_SIZE):
        emb_batch_set.append(org_text_emb)
        cod_batch_set.append(ko_grammar_set[BATCH_SIZE*i+j])

    emb_batch_set = np.asarray(emb_batch_set)
    cod_batch_set = np.asarray(cod_batch_set)
    dataset.append((emb_batch_set,cod_batch_set))

print(f'Total dataset count :{BATCH_COUNT*BATCH_SIZE}')


Text embedding shape : (1024,)
Total dataset count :3200


In [23]:
# 30번째 배치의 형태소코드셋 중 20번째꺼 확인
print(dataset[30][1][20])
print(dataset[31][1][20])

[2 9 2 5 2 2 5 6 2 2 5 6 2 2 5 2 5 6 3 2 5 2 5 2 6 2 5 6 2 5 6 6 2 8 3 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[ 2  2  5  2  5  2  5  2  2  2  5  2  8  3  2  5  2  6  6  2  5  2  5  3
 11  3 11  3 11  3 11  3 11  3 11  3 11  3  2  2  5  2  5  6  2  6  2  5
  6  3  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0]


# Generator 구현

In [228]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts([org_text])
total_words = len(tokenizer.word_index)

print(f'Total token count of origin text : {total_words}')

Total token count of origin text : 394


원문을 40개의 token 으로 구성된 문장으로 요약 하는것....<br>
향후 LSTM으로 G 구성 필요

In [229]:

_MAX_GEN_TOKEN = 40
_NOISE_DIM = total_words

word_table = {}

for word,index in tokenizer.word_index.items():
    word_table[index-1] = word

current_token_len = len(word_table)



In [230]:
print('Token table of origin text')
print('---------------------------------------------')
print(' Code         Token      ')
print('')
for k in word_table.keys():
  print( f'  {str(k).ljust(8)}    {word_table[k]}')
print('---------------------------------------------')


Token table of origin text
---------------------------------------------
 Code         Token      

  0           원내대표는
  1           주
  2           것
  3           이라고
  4           며
  5           문재인
  6           않는
  7           문
  8           이라며
  9           이제
  10          풀들이
  11          수
  12          대통령과
  13          공수처는
  14          될
  15          있는
  16          향해
  17          보인다
  18          짓밟힌
  19          국민적
  20          저항에
  21          직면할
  22          정권이
  23          위해
  24          공수처법을
  25          제게
  26          대통령은
  27          '공수처는
  28          없는
  29          야당이
  30          공수처장
  31          사람들이
  32          민주당
  33          그러면서
  34          공수처가
  35          청와대와
  36          왜
  37          그
  38          인정하지
  39          이
  40          공수처를
  41          권력
  42          게
  43          정권의
  44          민주당이
  45          국회에서
  46          그걸
  47          권력은
  48          먼저
  49          주호영
  50        

In [29]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import (Input,
                                     Dense, 
                                     BatchNormalization, 
                                     LeakyReLU,
                                     Softmax,
                                     Reshape, 
                                     Conv2DTranspose,
                                     Conv2D,
                                     Dropout,
                                     Flatten,
                                     Concatenate,
                                     Lambda)
import matplotlib.pyplot as plt

noise를 token_table을 통해 text로 변환하는 변환기 구현

In [234]:
import sys

def to_text(w):

    #tf.print(w)
    texts = []
    try:
        s_w = tf.sort(w,axis=-1,direction='DESCENDING')
        #print(s_w)
        for w_0r,s_w_0r in zip(w,s_w):
            text = ""
            wh = tf.where(tf.math.greater(w_0r,s_w_0r[_MAX_GEN_TOKEN]))

            for k in tf.reshape(wh,(_MAX_GEN_TOKEN,)).numpy():
                try:
                    text += word_table[k]+' '
                except Exception as ex:
                    tf.print('to_text : word_table' + str(ex),k)
                
            texts.append(text)
    except Exception as ex:
        tf.print('to_text : ' + str(ex),sys.exc_info())

    return tf.constant(texts,dtype=tf.string)

In [236]:
# to_text 함수의 test
w = tf.random.normal([3,_NOISE_DIM])
print(w.shape)
e = to_text(w)
for t in e:
    print(t.numpy().decode('utf-8'))

(3, 394)
공수처는 공수처가 게 그걸 등을 자신의 지적했다 표정으로 상임이사국처럼 있는데 전 토대가 대통령이 윤건영 이권을 반대하는지 지금까지 법치가 무오류의 오류를 추종자들로 넘쳐 전락할 법조인이지만 할지 법으로 독립성을 검찰총장을 운영할지 수사를 추미애보다 더 180석의 군사작전을 막겠는가 풀이다 풀을 시인 일어나는' 일어서서 
공수처는 향해 위해 '공수처는 없는 사람들이 공수처를 22일 경고했다 놓고 만들어낸 해보지 않고 출범할 상임이사국처럼 시비를 것이라고 윤건영 의원이 언급했다 불거져도 가져가 않았는가 오류를 자신들의 지킴으로서 난다 지도자의 신성을 마음대로 사법기구가 추미애 저지하려다가 써서라도 또 온통 바람 불면 밭의 못하도록 
이라며 위해 민주당이 먼저 가덕도 등을 엄포를 이어 표정으로 없이는 행사할 '민주당 것은 민주주의를 거리낌없이 공무원 앉아 우려했다 분이 주변을 지금까지 비리의 비판했다 저도 공수처장이 수사관들을 위에 헌법과 핍박하는 일 파렴치와 겪어온 행태가 라고 삭발하고 이용한다는 아울러 짓밟지만 민초의 광장에서 


생성된 text의 embedding 변환기 구현<br>
embedding은 org_text의 embedding과 비교하여 원문과 유사하게 민들기 위한 목적

In [63]:
# 이거 이틀걸림...잘 몰라서 ㅈㄴ 헤맴

@tf.custom_gradient
def to_embedding(w):

    def grad(dy):
        dy = tf.nn.batch_normalization(dy, 0, tf.math.reduce_std(dy), 0, 4, 1e-3)  # gradient 증폭 
        dy_arr = tf.reshape(dy,(dy.shape[0],1024,1))
        #tf.print(dy_arr)
        dy_arr_st = tf.image.resize(dy_arr, size=(dy.shape[0],_MAX_LENGTH*_MAX_TOKEN),method=tf.image.ResizeMethod.BILINEAR)
        dy_arr_st = tf.reshape(dy_arr_st,shape=(dy.shape[0],_MAX_LENGTH,_MAX_TOKEN))
        return dy_arr_st

    #print(w)    
    texts = []
    value = None
    texts = []
    try:
        s_w = tf.sort(w,axis=-1,direction='DESCENDING')
        #print(s_w)
        for w_0r,s_w_0r in zip(w,s_w):
            text = ""
            wh = tf.where(tf.math.greater(w_0r,s_w_0r[_MAX_GEN_TOKEN]))
            for k in tf.reshape(wh,(_MAX_GEN_TOKEN,)).numpy():
                text += word_table[k]+' '
                 
            texts.append(text)                 
        value = tf.constant(embedder.encode(texts,show_progress_bar=False),dtype=tf.float32)
    except Exception as ex:
        tf.print(ex,sys.exc_info())

    return value, grad

In [71]:
# to_embedding 함수의 test
e = to_embedding(w)
for t in e:
    print(t.numpy())

[ 0.49004325 -0.2691142   0.6026438  ... -0.16845913 -0.03569949
 -0.00082758]
[ 0.35038516 -0.4458071   0.5340477  ... -0.5159919  -0.03732621
  0.25392553]


In [147]:
# to_compression_ratio

@tf.custom_gradient
def to_compression_ratio(w):
    def grad(dy):
        dy = tf.nn.batch_normalization(dy, 0, tf.math.reduce_std(dy), 0, 4, 1e-3)  # gradient 증폭, 이래도 되는 건지....
        dy_arr = tf.reshape(dy,(dy.shape[0],1,1))
        #tf.print(dy_arr)
        dy_arr_st = tf.image.resize(dy_arr, size=(dy.shape[0],_MAX_LENGTH*_MAX_TOKEN),method=tf.image.ResizeMethod.BILINEAR)
        dy_arr_st = tf.reshape(dy_arr_st,shape=(dy.shape[0],_MAX_LENGTH,_MAX_TOKEN))        
        return dy_arr_st

    const = tf.math.reduce_sum(tf.constant([x for x in range(tf.size(w[0]))],tf.int64)) 
    #print(const)
    value = None
    compression_ratio = [] #tf.Variable()
    texts = []
    try:
        s_w = tf.sort(w,axis=-1,direction='DESCENDING')
        #print(s_w)
        for w_0r,s_w_0r in zip(w,s_w):
            wh = tf.where(tf.math.greater(w_0r,s_w_0r[_MAX_GEN_TOKEN]))
            cr = tf.math.reduce_sum(tf.reshape(wh,(_MAX_GEN_TOKEN,)))
            compression_ratio.append(cr.numpy()/const.numpy())
        #print(compression_ratio)
        value = tf.constant(compression_ratio,dtype=tf.float32)
    except Exception as ex:
        tf.print(ex,sys.exc_info())

    return value, grad
    

In [148]:
# to_compression_ratio 함수의 test
e = to_compression_ratio(w)
for t in e:
    print(t.numpy())

0.09176894
0.10754996


생성된 text의 morpheme code 변환기 구현<br>
morpheme code는 한국어 문장들(dataset)의 morpheme code와 비교하여 한국어 문법에 가깝게 만들기 위한 목적

In [149]:

@tf.custom_gradient
def to_morpcoding(w):

    def grad(dy):
        dy = tf.nn.batch_normalization(dy, 0, tf.math.reduce_std(dy), 0, 4, 1e-3)  # gradient 증폭, 이래도 되는 건지....
        dy_arr = tf.reshape(dy,(dy.shape[0],_MAX_MORP_LENGTH,1))
        #tf.print(dy_arr)
        dy_arr_st = tf.image.resize(dy_arr, size=(dy.shape[0],_MAX_LENGTH*_MAX_TOKEN),method=tf.image.ResizeMethod.BILINEAR)
        dy_arr_st = tf.reshape(dy_arr_st,shape=(dy.shape[0],_MAX_LENGTH,_MAX_TOKEN))
        return dy_arr_st

    #print(w)    
    codes = []
    value = None
    texts = []
    try:
        s_w = tf.sort(w,axis=-1,direction='DESCENDING')
        #print(s_w)
        for w_0r,s_w_0r in zip(w,s_w):
            text = ""
            wh = tf.where(tf.math.greater(w_0r,s_w_0r[_MAX_GEN_TOKEN]))
            for k in tf.reshape(wh,(_MAX_GEN_TOKEN,)).numpy():
                text += word_table[k]+' '
                 
            texts.append(text)    
            
        for sentence in texts:
            code = morpheme_encode(sentence)
            if len(code) <= _MAX_MORP_LENGTH:
                codes.append(code + [_PADDING_CODE for i in range(_MAX_MORP_LENGTH-len(code))])
            else:
                codes.append(code[:_MAX_MORP_LENGTH])
        value = tf.constant(codes,dtype=tf.int32)
    except Exception as ex:
        tf.print(ex,sys.exc_info())

    return value, grad

In [150]:
# to_morpcoding 함수의 test
e = to_morpcoding(w)
for t in e:
    print(t.numpy())

[ 2  2  9  2  5  2  5  7  7  2  2  5  7  7  2  2  5  2  6  3  2  2  3  5
  6  2  2  3  5  2  5  6  2  3  2  2  2  9  5  3  7  2  3  2  2  5  2  5
  2  5  7  2  5  2  5  6  2  6  2  5  7  7  2  9  2  9  5  2  6  6  2 10
  8  8  2  5  6  6  2  2  5  6  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0]
[ 2  2  2  9  2  2  2  2  2  5  7  7  2  6  2  5  2  5  2  2  6  8  2  5
  3  2  2  5  2  2  2  2  2  2  6  2  7  2  9  5  2  2  5  2  5 11  2  5
  2  8  2  5  2  2  2  2  5  2  5  2  6  2  5  2  2  5  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0]


Network 구성을 위해 사용자 정의 Layer 를 구현

In [151]:
# 이것도 잘 몰라서 하루 걸림... ㅜㅜ

from keras import backend as K
from keras.layers import Layer

#tf.executing_eagerly()

class Post_processing(Layer):

    def __init__(self, output_dim, encoder_func=None,Tout=tf.float32, **kwargs):
        self.output_dim = output_dim
        self.encoder = encoder_func
        self.Tout = Tout
        super(Post_processing, self).__init__(**kwargs)
    '''
    def build(self, input_shape):
        tf.print('build',input_shape)
        # 이 레이어에 대해 학습가능한 가중치 변수를 만듭니다.
        self.kernel = self.add_weight(name='kernel', 
                                      shape=(input_shape[1], self.output_dim),
                                      initializer='uniform',
                                      trainable=True)
        super(Post_processing, self).build(input_shape)  # 끝에서 꼭 이 함수를 호출하십시오
    '''
    def call(self, input_data):
        #tf.print('Post_processing : call input_data',input_data.shape)
        value = tf.py_function(self.encoder,[input_data],Tout=self.Tout,name='encode_func')
        #print('value.shape:',value.shape)
        #value.set_shape((input_data.shape[0],self.output_dim))
        if self.output_dim > 0:
            value.set_shape((input_data.shape[0],self.output_dim))
        else:
            value.set_shape((input_data.shape[0],))
        #return tf.reshape(value,[input_data.shape[0]])  

        #value = tf.Variable((tf.zeros([input_data.shape[0],1024]) if self.Tout==tf.float32 else tf.zeros([input_data.shape[0],])),dtype=self.Tout,shape=( (input_data.shape[0],1024) if self.Tout==tf.float32 else (input_data.shape[0],)))
        #tf.py_function(self.encoder,[input_data],Tout=self.Tout)
        return value

    def compute_output_shape(self, input_shape):
        tf.print('compute_output_shape:',input_shape)
        input_shape = tensor_shape.TensorShape(input_shape).as_list()
        if self.output_dim > 0:
            return tensor_shape.TensorShape([input_shape[0], self.output_dim])
        return tensor_shape.TensorShape([input_shape[0]])

In [152]:
# 구성한 Layer의 test
e = Post_processing(1024,to_embedding,Tout=tf.float32)(w)
for c in e:
    print(c)

tf.Tensor(
[ 0.49004325 -0.2691142   0.6026438  ... -0.16845913 -0.03569949
 -0.00082758], shape=(1024,), dtype=float32)
tf.Tensor(
[ 0.35038516 -0.4458071   0.5340477  ... -0.5159919  -0.03732621
  0.25392553], shape=(1024,), dtype=float32)


In [153]:
# 별로 중요하지는 않지만 Lambda layer를 활용하기 위한 assert 함수 구성
def assert_layer(input_data,out_dim=None):
    #tf.print(input_data)
    #print(input_data)
    assert input_data.shape[1] == out_dim
    return input_data

In [237]:
# 드디어 generator 구현
# 효과적으로 구성된 것인지는 모르겠음... 이것은 아직 많은 연구가 필요함.
# 또한 LSTM으로 바꾸어 길이의 한게를 극복해야 할 것...

def make_generator_model(org_words):
    input = Input(shape=(org_words,), dtype='float32') 
    x1 = Dense(org_words*2, use_bias=False)(input)
    #x1 = BatchNormalization()(x1)
    x1 = LeakyReLU()(x1)
    
    x1 = Dense(org_words*4, use_bias=False)(x1)
    #x1 = BatchNormalization()(x1)
    x1 = LeakyReLU()(x1)
    
    #x1 = Dense(max_length*total_words, use_bias=False, activation='tanh')(x1)
    x1 = Dense(org_words, use_bias=False)(x1)
    x1 = Lambda(assert_layer,arguments={'out_dim':org_words})(x1)
    #x1 = Reshape((max_length, total_words))(x1)
    #x1 = BatchNormalization()(x1)
    #x1 = Softmax()(x1)        
    #x1 = MyCustomLayer(max_length*total_words)(x1)
    txt = Post_processing(0,to_text,Tout=tf.string)(x1)
    emb = Post_processing(1024,to_embedding,Tout=tf.float32)(x1)
    cmr = Post_processing(0,to_compression_ratio,Tout=tf.float32)(x1)
    cod = Post_processing(128,to_morpcoding,Tout=tf.int32)(x1)
    
    model = Model(input,[txt,emb,cmr,cod])
    
    model.summary()
    return model

generator = make_generator_model(_NOISE_DIM)

Model: "functional_11"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_10 (InputLayer)           [(None, 394)]        0                                            
__________________________________________________________________________________________________
dense_19 (Dense)                (None, 788)          310472      input_10[0][0]                   
__________________________________________________________________________________________________
leaky_re_lu_14 (LeakyReLU)      (None, 788)          0           dense_19[0][0]                   
__________________________________________________________________________________________________
dense_20 (Dense)                (None, 1576)         1241888     leaky_re_lu_14[0][0]             
______________________________________________________________________________________

In [238]:
# generator의 test
# Create a random noise and generate a sample
noise = tf.random.normal([3,_NOISE_DIM])
texts,embeddings,compratios,morpcodes = generator(noise, training=True)
print(texts.shape)
for i,txt in zip(range(len(texts)),texts):
    print(f" {i+1}> {txt.numpy().decode('utf-8')}" )
print(embeddings.shape)
print(compratios.shape)
print(morpcodes.shape)

(3,)
 1> 며 공수처법을 야당이 공수처장 민주당 민주당이 22일 가덕도 정의당을 꼼수 만들어낸 원내대표인 출범할 유엔 비난을 완성된다 것은 없이 챙기는 이해할 없다'고 감시하는 쓰레기 화신이 존재가 자신들의 지도자를 두렵기만 일 법무장관을 자리에는 편을 군사작전을 눈 아닌 풀을 다시 시인 마지막으로 광장에서 
 2> 짓밟힌 국회에서 권력은 건설 밀어붙이고 꼼수 선거법에 불법·탈법으로 유엔 '비토권'을 걱정하느냐고 거짓말이라는 비난을 공약으로 민주주의를 부정하는 군림하는 재정 괴물 앉아 챙기는 권력자들에 가져가 않았는가 바로 지도자를 전락할 질타했다 검찰과 세워 윤석열 봉착하자 편을 파렴치와 보일 대란대치 온통 바람 다시 일어서서 
 3> 이제 수 될 대통령은 공수처장 그러면서 경고했다 끌어들이기 안보리 거는 것은 출신 요직에 권력자들에 버리면 것인데 대통령 않았는가 정부를 무오류의 지도자를 수단으로 할지 앞장 검찰의 봉착하자 수를 자리에는 편을 막을 없다 봐야 온통 유지에 바람이 풀을 일어난다 시인 민초의 정권은 
(3, 1024)
(3,)
(3, 128)


# Discriminator 구현

먼저 요약을 구분하기 위한 discriminator_summ 구현

In [159]:
# 문장에 대한 embeddings를 이용하여 org_text_emb (org_text의 embedding)과의 유사도를 계산한다.
import scipy

@tf.custom_gradient
def to_similarity(w):

    def grad(dy):
        dy = tf.nn.batch_normalization(dy, 0, tf.math.reduce_std(dy), 0, 10, 1e-3)  # gradient 증폭, 이래도 되는 건지....
        dy_arr = tf.reshape(dy,(dy.shape[0],1,1))
        #tf.print(dy_arr)
        dy_arr_st = tf.image.resize(dy_arr, size=(dy.shape[0],1024),method=tf.image.ResizeMethod.BILINEAR)
        dy_arr_st = tf.reshape(dy_arr_st,shape=(dy.shape[0],1024))
        return dy_arr_st

    similarities = []
    value = None
    try:
        for embedding in w:
            distances = scipy.spatial.distance.cdist([embedding], [org_text_emb], "cosine")[0]
            similarities.append(distances[0])
            
        value = tf.constant(similarities,dtype=tf.float32)
    except Exception as ex:
        tf.print(ex,sys.exc_info())

    return value, grad


In [239]:
def make_discriminator_model():
    input_emb = Input(shape=(1024,), dtype='float32') 
    #x1 = Post_processing(0,to_similarity,Tout=tf.float32)(input_emb)
    #x1 = Reshape((1,))(x1)    
    x1 = Dense(1024*2)(input_emb)
    x1 = BatchNormalization()(x1)
    x1 = LeakyReLU()(x1)

    
    input_cmp = Input(shape=(), dtype='float32') 
    imput_mrp = Input(shape=(_MAX_MORP_LENGTH,), dtype='int32')

    x2 = Reshape((1,))(input_cmp)
    x3 = Dense(256)(imput_mrp)
    #x3 = BatchNormalization()(x3)
    x3 = LeakyReLU()(x3)

    x3 = Dense(256*3)(x3)
    #x3 = BatchNormalization()(x3)
    x3 = LeakyReLU()(x3)

    concatted = Concatenate(axis=1)([x1, x2, x3])
    x4 = Flatten()(concatted)
    x4 = Dense(64, use_bias=False)(x4)
    #x4 = BatchNormalization()(x4)
    x4 = LeakyReLU()(x4)
    x4 = Dense(1)(x4)
    
    model = Model([input_emb,input_cmp,imput_mrp],x4)
    
    model.summary()
    
    return model

discriminator = make_discriminator_model()

Model: "functional_13"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_13 (InputLayer)           [(None, 128)]        0                                            
__________________________________________________________________________________________________
input_11 (InputLayer)           [(None, 1024)]       0                                            
__________________________________________________________________________________________________
dense_23 (Dense)                (None, 256)          33024       input_13[0][0]                   
__________________________________________________________________________________________________
dense_22 (Dense)                (None, 2048)         2099200     input_11[0][0]                   
______________________________________________________________________________________

In [240]:
# discriminator test

predict = discriminator([embeddings,compratios,morpcodes])
print(predict)

tf.Tensor(
[[0.14201309]
 [0.06003584]
 [0.07988645]], shape=(3, 1), dtype=float32)


# GAN 을 이용한 loss 의 gradient 구현 --> 빡심

In [241]:
# This method returns a helper function to compute cross entropy loss
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)

def discriminator_loss(real_output, fake_output):
    real_loss = cross_entropy(tf.ones_like(real_output), real_output)
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    total_loss = real_loss + fake_loss
    return total_loss

def generator_loss(fake_output):
    return cross_entropy(tf.ones_like(fake_output), fake_output)

generator_optimizer = tf.keras.optimizers.Adam(1e-4)
discriminator_optimizer = tf.keras.optimizers.Adam(1e-4)

In [242]:
# 디짐
@tf.function
def train_step(real_embedding,real_morpcoding):
  
    # 1 - Create a random noise to feed it into the model
    # for the text generation
    noise = tf.random.normal([BATCH_SIZE, _NOISE_DIM])
    
    # 2 - Generate text and calculate loss values
    # GradientTape method records operations for automatic differentiation.
    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        texts,embeddings,compratios,morpcodes = generator(noise, training=True)
        #embeddings = generator(noise, training=True)
        # real에 가까우려면 discriminator의 학습은 real_embedding 이 zero (0)에 가깝게 학습시켜야 함.
        # 하지만 압축율의 개념으로는 본래는 ones (1)가 맞음.
        real_output = discriminator([real_embedding,np.zeros(len(real_embedding)),real_morpcoding], training=True)
        fake_output = discriminator([embeddings,compratios,morpcodes], training=True)

        #tf.print('train_step : embeddings.shape=',embeddings.shape)
        gen_loss = generator_loss(fake_output)
        #tf.print('train_step : gen_loss=',gen_loss)
        disc_loss = discriminator_loss(real_output, fake_output)
        #tf.print('train_step : disc_loss=',disc_loss)

    # 3 - Calculate gradients using loss values and model variables
    # "gradient" method computes the gradient using 
    # operations recorded in context of this tape (gen_tape and disc_tape).
    
    # It accepts a target (e.g., gen_loss) variable and 
    # a source variable (e.g.,generator.trainable_variables)
    # target --> a list or nested structure of Tensors or Variables to be differentiated.
    # source --> a list or nested structure of Tensors or Variables.
    # target will be differentiated against elements in sources.

    # "gradient" method returns a list or nested structure of Tensors  
    # (or IndexedSlices, or None), one for each element in sources. 
    # Returned structure is the same as the structure of sources.
    
    gradients_of_discriminator = disc_tape.gradient(disc_loss, 
                                                discriminator.trainable_variables)
    #tf.print('train_step : gradients_of_discriminator=',gradients_of_discriminator)   
    noise = tf.random.normal([BATCH_SIZE, _NOISE_DIM])
    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        texts,embeddings,compratios,morpcodes = generator(noise, training=True)
        #embeddings = generator(noise, training=True)
        #real_output = discriminator(real_embedding, training=True)
        fake_output = discriminator([embeddings,compratios,morpcodes], training=True)

        #tf.print('train_step : embeddings.shape=',embeddings.shape)
        gen_loss = generator_loss(fake_output)
        #tf.print('train_step : gen_loss=',gen_loss)
        #disc_loss = discriminator_loss(real_output, fake_output)
        #tf.print('train_step : disc_loss=',disc_loss)

    gradients_of_generator = gen_tape.gradient(gen_loss, 
                                               generator.trainable_variables)
    #tf.print('train_step : gradients_of_generator=',gradients_of_generator)
 

    # 4 - Process  Gradients and Run the Optimizer
    # "apply_gradients" method processes aggregated gradients. 
    # ex: optimizer.apply_gradients(zip(grads, vars))
    """
    Example use of apply_gradients:
    grads = tape.gradient(loss, vars)
    grads = tf.distribute.get_replica_context().all_reduce('sum', grads)
    # Processing aggregated gradients.
    optimizer.apply_gradients(zip(grads, vars), experimental_aggregate_gradients=False)
    """
    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))
    #tf.print('train_step : after discriminator_optimizer')    

In [243]:
EPOCHS = 5
# 요약문 생성의 확인을 위해 10개의 문장을 생성하고 train과정에서 각 epoch마다 변화를 확인한다.
seed = tf.random.normal([10, _NOISE_DIM])

In [245]:
# 생성된 문장의 원문 유사도를 측정하기 위한 함수

import scipy
#print(doc_emb)
def similarity_score(queries,org_embedding):

    total_score = 0
    query_embeddings = embedder.encode(queries,show_progress_bar=False)
    for query, query_embedding in zip(queries, query_embeddings):
        distances = scipy.spatial.distance.cdist([query_embedding], [org_embedding], "cosine")[0]
        results = zip(range(len(distances)), distances)
        for idx, distance in results:
            total_score += 1-distance
    return total_score

queries = []

texts,embeddings,compratios,morpcodes = generator(seed,training=False)

#count = 0
for t in texts:
    summary_text = t.numpy().decode('utf-8')
    queries.append(summary_text)
print('Similarity score:',str(similarity_score(queries,org_text_emb)))


Similarity score: 6.930801350407938


In [246]:
# Print iterations progress
class ProgressBar:
    # pb = ProgressBar(total=20, prefix = 'Epoch 1')
    def __init__(self,total=20, prefix = '', suffix = '', decimals = 1, length = 20, fill = '█', printEnd = "\r"):
        self.total = total
        self.prefix = prefix
        self.suffix = suffix
        self.decimals = decimals
        self.length = length
        self.fill = fill
        self.printEnd = printEnd
        self.ite = 0
    # pb.printProgress(1,'~~~~')
    def printProgress(self,iteration, text):
        self.ite += iteration
        percent = ("{0:." + str(self.decimals) + "f}").format(100 * (self.ite / float(self.total)))

        filledLength = int(self.length * self.ite // self.total)
        bar = self.fill * filledLength + '-' * (self.length - filledLength)
        print(f'\r{self.prefix} |{bar}| {percent}% {self.suffix}  {text}', end="", flush=True)
        # Print New Line on Complete
        if self.ite == self.total: 
            print()

In [247]:
import time
from IPython import display # A command shell for interactive computing in Python.
import re

def train(dataset, epochs):
    print('Start with seed text.')
    seed = tf.random.normal([10, _NOISE_DIM])
    print('-------------------------------------------------------')
    texts,embeddings,compratios,morpcodes = generator(seed,training=False)
    for i,t in  zip(range(len(texts)),texts):
        summary_text = t.numpy().decode('utf-8')
        print(f'{i+1} > {summary_text}')        
    print('-------------------------------------------------------')
    print('')

    # A. For each epoch, do the following:
    for epoch in range(epochs):
        start = time.time()
        pb = ProgressBar(total=BATCH_COUNT, prefix = f'Epoch:{str(epoch+1)}/{epochs}')
        pb.printProgress(0,'Start batch.')
        # 1 - For each batch of the epoch, 
        for batch_num,(emb_batch_set,cod_batch_set) in zip(range(len(dataset)),dataset):
            # 1.a - run the custom "train_step" function
            # we just declared above
            #print(image_batch.shape)
            train_step(emb_batch_set,cod_batch_set)
            texts,embeddings,compratios,morpcodes = generator(seed,training=False)
            pb.printProgress(+1,f"Time for batch {batch_num + 1}/{BATCH_COUNT} is {int(time.time()-start)} sec, generated text:{texts[0].numpy().decode('utf-8')}")
        # 4 - Print out the completed epoch no. and the time spent
        #print (f'Time for epoch {} is {} sec'.format(epoch + 1, time.time()-start))
        #texts,embeddings,compratios,morpcodes = generator(seed,training=False)
        count = 0
        queries = []
        texts,embeddings,compratios,morpcodes = generator(seed,training=False)
        for i,t in  zip(range(len(texts)),texts):
            summary_text = t.numpy().decode('utf-8')
            print(f'{i+1} > {summary_text}')
            queries.append(summary_text)
            c = [m.start() for m in re.finditer(_MISMATCH_WORD, summary_text)]
            count += len(c)
        print(f'Mismatch count:{count} Similarity score:{str(similarity_score(queries,org_text_emb))}')
        print('')

In [ ]:
EPOCHS = 5
train(dataset, EPOCHS)